In [1]:
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
from model import create_model
from align import AlignDlib
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import sys
from PIL import Image
sys.modules['Image'] = Image
from PIL import Image
import warnings
from scipy import ndimage
warnings.filterwarnings('ignore')

In [3]:
# Model for 128 embeddings from a image
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [4]:
# Model Summary
# nn4_small2_pretrained.summary()

In [5]:
# Initialize the OpenFace face alignment utility
alignment = AlignDlib('weights/landmarks.dat')

In [6]:
# Funtion to align an image using dlib utility
def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [7]:
def load_image(path):
    img = cv2.imread(path, 1)
    return img[...,::-1]

In [8]:
class IdentityMetadata():
    def __init__(self, base, name, file):
        self.base = base
        self.name = name
        self.file = file

    def __repr__(self):
        return self.image_path()
    
    def subdir_path(self):
        return self.name
    
    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path):
    metadata = []
    for i in sorted(os.listdir(path)):
        for f in sorted(os.listdir(os.path.join(path, i))):
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

metadata = load_metadata('images')

In [9]:
datagen = ImageDataGenerator(rotation_range=15, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1,
                             shear_range=0.15, 
                             zoom_range=0.1,
                             channel_shift_range = 10, 
                             brightness_range=[0.2,1.0],
                             horizontal_flip=True)
import os

folder_name='AugmentedImages/'
save_here = 'C:/Users/hp/Desktop/aastha/face-recognitition-ams/'+folder_name

if not os.path.exists(save_here):
    os.makedirs(save_here) 
    for i, m in enumerate(metadata): 
        img = load_image(m.image_path())
        datagen.fit(np.expand_dims(img,0))
        directory=save_here+m.subdir_path()
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(directory)
        for x, val in zip(datagen.flow(np.expand_dims(img,0), 
                                       save_to_dir=directory, 
                                       save_prefix='AugImage', 
                                       save_format='jpg'),range(5)) :   
            pass

C:/Users/hp/Desktop/aastha/face-recognitition-ams/AugmentedImages/Aastha_Agrawal
C:/Users/hp/Desktop/aastha/face-recognitition-ams/AugmentedImages/Abhishek_Ranjan
C:/Users/hp/Desktop/aastha/face-recognitition-ams/AugmentedImages/Anjali_Agarwal
C:/Users/hp/Desktop/aastha/face-recognitition-ams/AugmentedImages/Anmol_Jain
C:/Users/hp/Desktop/aastha/face-recognitition-ams/AugmentedImages/Arunima_Mitra
C:/Users/hp/Desktop/aastha/face-recognitition-ams/AugmentedImages/Dev_Pabreja
C:/Users/hp/Desktop/aastha/face-recognitition-ams/AugmentedImages/Josika_Kar
C:/Users/hp/Desktop/aastha/face-recognitition-ams/AugmentedImages/Rahmeen_Habib
C:/Users/hp/Desktop/aastha/face-recognitition-ams/AugmentedImages/Swapan_Asija


In [10]:
metadata1 = load_metadata('AugmentedImages')

In [11]:
# To store image embeddings
embedded = np.zeros((metadata1.shape[0], 128))

# Process image through the neural net to get 128 embeddings per image
for i, m in enumerate(metadata1):    
    print(i, " ",m)
    img = load_image(m.image_path())
    img = align_image(img)
    if img is None:
#         print("No face found")
        continue
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    # obtain embedding vector for image
    embedded[i] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]

0   AugmentedImages\Aastha_Agrawal\AugImage_0_105.jpg

1   AugmentedImages\Aastha_Agrawal\AugImage_0_1438.jpg
2   AugmentedImages\Aastha_Agrawal\AugImage_0_1466.jpg
3   AugmentedImages\Aastha_Agrawal\AugImage_0_1515.jpg
4   AugmentedImages\Aastha_Agrawal\AugImage_0_1609.jpg
5   AugmentedImages\Aastha_Agrawal\AugImage_0_1718.jpg
6   AugmentedImages\Aastha_Agrawal\AugImage_0_18.jpg
7   AugmentedImages\Aastha_Agrawal\AugImage_0_1900.jpg
8   AugmentedImages\Aastha_Agrawal\AugImage_0_2647.jpg
9   AugmentedImages\Aastha_Agrawal\AugImage_0_2809.jpg
10   AugmentedImages\Aastha_Agrawal\AugImage_0_2818.jpg
11   AugmentedImages\Aastha_Agrawal\AugImage_0_3018.jpg
12   AugmentedImages\Aastha_Agrawal\AugImage_0_3032.jpg
13   AugmentedImages\Aastha_Agrawal\AugImage_0_3598.jpg
14   AugmentedImages\Aastha_Agrawal\AugImage_0_3678.jpg
15   AugmentedImages\Aastha_Agrawal\AugImage_0_3683.jpg
16   AugmentedImages\Aastha_Agrawal\AugImage_0_3867.jpg
17   AugmentedImages\Aastha_Agrawal\AugImage_0_4033.jpg
18  

142   AugmentedImages\Anjali_Agarwal\AugImage_0_528.jpg
143   AugmentedImages\Anjali_Agarwal\AugImage_0_5364.jpg
144   AugmentedImages\Anjali_Agarwal\AugImage_0_5523.jpg
145   AugmentedImages\Anjali_Agarwal\AugImage_0_5532.jpg
146   AugmentedImages\Anjali_Agarwal\AugImage_0_5780.jpg
147   AugmentedImages\Anjali_Agarwal\AugImage_0_5800.jpg
148   AugmentedImages\Anjali_Agarwal\AugImage_0_5987.jpg
149   AugmentedImages\Anjali_Agarwal\AugImage_0_653.jpg
150   AugmentedImages\Anjali_Agarwal\AugImage_0_6595.jpg
151   AugmentedImages\Anjali_Agarwal\AugImage_0_6622.jpg
152   AugmentedImages\Anjali_Agarwal\AugImage_0_6654.jpg
153   AugmentedImages\Anjali_Agarwal\AugImage_0_6783.jpg
154   AugmentedImages\Anjali_Agarwal\AugImage_0_6896.jpg
155   AugmentedImages\Anjali_Agarwal\AugImage_0_7629.jpg
156   AugmentedImages\Anjali_Agarwal\AugImage_0_7673.jpg
157   AugmentedImages\Anjali_Agarwal\AugImage_0_7854.jpg
158   AugmentedImages\Anjali_Agarwal\AugImage_0_7877.jpg
159   AugmentedImages\Anjali_Agar

291   AugmentedImages\Dev_Pabreja\AugImage_0_1716.jpg
292   AugmentedImages\Dev_Pabreja\AugImage_0_1892.jpg
293   AugmentedImages\Dev_Pabreja\AugImage_0_2040.jpg
294   AugmentedImages\Dev_Pabreja\AugImage_0_2084.jpg
295   AugmentedImages\Dev_Pabreja\AugImage_0_218.jpg
296   AugmentedImages\Dev_Pabreja\AugImage_0_2221.jpg
297   AugmentedImages\Dev_Pabreja\AugImage_0_2256.jpg
298   AugmentedImages\Dev_Pabreja\AugImage_0_2366.jpg
299   AugmentedImages\Dev_Pabreja\AugImage_0_2387.jpg
300   AugmentedImages\Dev_Pabreja\AugImage_0_279.jpg
301   AugmentedImages\Dev_Pabreja\AugImage_0_2871.jpg
302   AugmentedImages\Dev_Pabreja\AugImage_0_3315.jpg
303   AugmentedImages\Dev_Pabreja\AugImage_0_3338.jpg
304   AugmentedImages\Dev_Pabreja\AugImage_0_3654.jpg
305   AugmentedImages\Dev_Pabreja\AugImage_0_3837.jpg
306   AugmentedImages\Dev_Pabreja\AugImage_0_3857.jpg
307   AugmentedImages\Dev_Pabreja\AugImage_0_3922.jpg
308   AugmentedImages\Dev_Pabreja\AugImage_0_4280.jpg
309   AugmentedImages\Dev_Pabr

443   AugmentedImages\Rahmeen_Habib\AugImage_0_9464.jpg
444   AugmentedImages\Rahmeen_Habib\AugImage_0_9557.jpg
445   AugmentedImages\Rahmeen_Habib\AugImage_0_9664.jpg
446   AugmentedImages\Rahmeen_Habib\AugImage_0_9753.jpg
447   AugmentedImages\Rahmeen_Habib\AugImage_0_986.jpg
448   AugmentedImages\Rahmeen_Habib\AugImage_0_9934.jpg
449   AugmentedImages\Rahmeen_Habib\AugImage_0_9949.jpg
450   AugmentedImages\Swapan_Asija\AugImage_0_1329.jpg
451   AugmentedImages\Swapan_Asija\AugImage_0_1340.jpg
452   AugmentedImages\Swapan_Asija\AugImage_0_1438.jpg
453   AugmentedImages\Swapan_Asija\AugImage_0_1525.jpg
454   AugmentedImages\Swapan_Asija\AugImage_0_1595.jpg
455   AugmentedImages\Swapan_Asija\AugImage_0_1764.jpg
456   AugmentedImages\Swapan_Asija\AugImage_0_1778.jpg
457   AugmentedImages\Swapan_Asija\AugImage_0_1817.jpg
458   AugmentedImages\Swapan_Asija\AugImage_0_2042.jpg
459   AugmentedImages\Swapan_Asija\AugImage_0_2078.jpg
460   AugmentedImages\Swapan_Asija\AugImage_0_2197.jpg
461 

In [63]:
from sklearn.preprocessing import LabelEncoder
targets = np.array([m.name for m in metadata1])
encoder = LabelEncoder()
encoder.fit(targets)

# Numerical encoding of identities
y = encoder.transform(targets)

In [64]:
train_idx = np.arange(metadata1.shape[0]) % 2 != 0
test_idx = np.arange(metadata1.shape[0]) % 2 == 0

# 50 train examples of 10 identities (5 examples each)
X_train = embedded[train_idx]
# 50 test examples of 10 identities (5 examples each)
X_test = embedded[test_idx]

y_train = y[train_idx]
y_test = y[test_idx]

In [128]:
### SUPPORT VECTOR CLASSIFIER
# svc = SVC(kernel="linear", C=0.025) #10 -> 57% accuracy, 30-> 31%, 50->45%
# svc = SVC(gamma=2, C=1, probability=True) #30-> 88%, 50->88%

### Gaussian CLASSIFIER
# svc = GaussianProcessClassifier(1.0 * RBF(1.0)) # 30->90%, 50->89%

### KNN CLASSIFIER
# svc = KNeighborsClassifier(2) #10 images -> 85% accuracy, 30,50 images->88%
# svc = KNeighborsClassifier(5) #10 -> 78% accuracy, 30-> 86%, 50->87%

### DECISION TREE CLASSIFIER
# svc = DecisionTreeClassifier(max_depth=5) # 30,50->58%
# svc = DecisionTreeClassifier(max_depth=10) # 30->68%, 50->79%
# svc = DecisionTreeClassifier(max_depth=20) #30-> 70%, 50->80%
# svc = DecisionTreeClassifier(max_depth=30) # 30->73%, 50->80%
# svc = DecisionTreeClassifier(max_depth=40) # 30->73%, 50->78%

### Random Forest Classifier
# svc = RandomForestClassifier(max_depth=10, n_estimators=50, max_features=1)  #10 -> 80% accuracy aprox, 30,50 -> 88%
# svc = RandomForestClassifier(40)  #10 -> 80% accuracy aprox, 30,50 -> 89%
# svc = RandomForestClassifier(30)  #10 -> 70% accuracy aprox, 30,50 -> 89%
# svc = RandomForestClassifier(20)  #10 -> 65% accuracy aprox, 30,50 -> 87%

### MLP CLASSIFIER
svc = MLPClassifier(alpha=1.0, max_iter=1000) # 30,50->88%

### ADA BOOST CLASSIFIER
# svc = AdaBoostClassifier(n_estimators=100, random_state=0) # 30->35%, 50->28%

### QUADRATIC CLASSIFIER
# svc = QuadraticDiscriminantAnalysis() #30->40%, 50->9%

#Naive bayes algorithm
# svc = GaussianNB() #10 -> 73% accuracy aprox, 30 -> 86%, 50 -> 88%

svc.fit(X_train, y_train)

acc_svc = accuracy_score(y_test, svc.predict(X_test))

print(f'SVM accuracy = {acc_svc}')

SVM accuracy = 0.8809523809523809


In [129]:
# train_idx = np.arange(metadata.shape[0]) 

# # 50 train examples of 10 identities (5 examples each)
# X_train = embedded[train_idx]

# y_train = y[train_idx]

# # svc = KNeighborsClassifier()
# # svc = SVC()
# svc = RandomForestClassifier(20)
# svc.fit(X_train, y_train)


In [130]:
# Prediction Code for live Images

In [131]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predClass = svc.predict([features])
    predClass = encoder.inverse_transform(predClass)[0]
    
    # Display the image with bounding box & label
    cv2.putText(origImg,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
    cv2.rectangle(origImg,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
    cv2.imshow('image',origImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [132]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame1(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predScores = svc.predict_proba([features])
    print(predScores)
    predClass = None
    
    maxProb = (predScores[0]).max()
    
    print(maxProb)
    
    if(maxProb >= 0.3):
    
        predClass = [(predScores[0]).argmax()]
        print(predClass)
        predClass = encoder.inverse_transform(predClass)[0]
    
    
    print(predClass)
    
    return bb, predClass

In [133]:
students = []

In [134]:
cap = cv2.VideoCapture(0)
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
#     print(ret, " ", frame)
    if frame is not None:  
        if(alignment.getLargestFaceBoundingBox(frame)!=None):
            bb, predClass = processFrame1(frame)
            cv2.rectangle(frame,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
            if(predClass!=None):
                cv2.putText(frame,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
                if predClass not in students:
                    students.append(predClass)
            cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

[[0.315811   0.00364922 0.31711928 0.09290348 0.03069645 0.0167023
  0.12754346 0.03410217 0.06147263]]
0.31711928330146383
[2]
Anjali_Agarwal
[[0.44772631 0.00313059 0.24477746 0.08052595 0.02366742 0.02408919
  0.09273478 0.02347816 0.05987014]]
0.44772631459571266
[0]
Aastha_Agrawal
[[0.28635162 0.00256721 0.36061379 0.1244453  0.0259768  0.01353949
  0.12793633 0.02337543 0.03519401]]
0.3606137945422148
[2]
Anjali_Agarwal
[[0.17233125 0.00571224 0.4197837  0.07965108 0.05407934 0.01381701
  0.18968792 0.03006815 0.0348693 ]]
0.4197837030785635
[2]
Anjali_Agarwal
[[0.18166869 0.00334316 0.39983613 0.09330106 0.0436141  0.00982916
  0.17500561 0.046223   0.0471791 ]]
0.39983612631464055
[2]
Anjali_Agarwal
[[0.21243671 0.0031158  0.37172661 0.09312262 0.0409448  0.01039848
  0.17062282 0.04075104 0.05688112]]
0.3717266091882196
[2]
Anjali_Agarwal
[[0.18260159 0.0026708  0.37955132 0.11737155 0.03253969 0.00791719
  0.19512012 0.04338762 0.03884012]]
0.37955132464760394
[2]
Anjali_Agar